# Brecha Salarial de Género en Argentina
Este proyecto fue realizado por el grupo 15 constituido por Daniela Ferraro y Jorgelina Sigal. Se utilizó un conjunto de datos provenientes del Ministerio de Trabajo, Empleo y Seguridad Social que contiene información sobre las remuneraciones en Argentina, desagregadas por género, tipo de industria y fecha. El objetivo es analizar las brechas salariales de género y su evolución. 

### Dataset
https://www.trabajo.gob.ar/estadisticas/oede/empleoyremuneracion.asp

### Contenido del dataset
#### Periodo: 
Indica año y mes al que corresponde la remuneración.
#### Industria: 
Nombre del sector o rama de actividad económica.
#### Género: 
Género del grupo de trabajadores al que pertenece la remuneración.
#### Mujer: 
Variable binaria (1 o 0) toma el valor 1 si es mujer, 0 si no. Útil para análisis estadístico, modelos y gráficos.
#### Varon: 
Variable binaria (1 o 0) toma el valor 1 si es varón, 0 si no. Útil para análisis estadístico, modelos y gráficos.
#### Remuneración: 
Remuneración promedio mensual para ese sector, género y período (en pesos argentinos).
#### Año:
Extracción del año de la columna período.

### Objetivo general
Analizar la evolución de la remuneración media del empleo registrado en el sector privado en Argentina, según rama de actividad y género, en el período comprendido entre enero de 1996 y octubre de 2022, utilizando herramientas de data science, con el fin de identificar patrones, desigualdades y tendencias vinculadas a la brecha salarial de género en el mundo laboral.

### Objetivos específicos
- Procesar y limpiar el conjunto de datos proporcionado mediante herramientas de data science, utilizando bibliotecas como pandas para su análisis exploratorio.

- Describir la evolución temporal de la remuneración media del empleo registrado en el sector privado, diferenciando por género y rama de actividad (según el CIIU rev. 3 a dos dígitos).

- Comparar las trayectorias salariales entre varones y mujeres en los distintos sectores de actividad, identificando las ramas con mayores y menores brechas de género.

- Visualizar los resultados mediante gráficos que faciliten la comprensión de las desigualdades salariales en el período analizado.

- Aplicar un modelo de aprendizaje supervisado de regresión para predecir la remuneración media en función del género, la rama de actividad y el período, evaluando su desempeño y analizando la importancia relativa de cada variable.

- Reflexionar sobre las implicancias sociales y económicas de las desigualdades de género en el mundo laboral formal, a partir de los hallazgos del análisis de datos.

### Tecnologías
- Python 3
- Pandas
- Jupyter Notebook
- Visual Studio Code
- Matplotlib 
- Seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

## Análisis y exploración de datos

In [ ]:
remuneraciones = pd.read_csv('Remuneraciones.csv', header=0, skiprows=[1])

In [ ]:
# Vista general del dataset
print(remuneraciones.head())

In [ ]:
print(remuneraciones.columns)

In [ ]:
# Eliminar espacios en nkmbres de columnas
remuneraciones.columns = remuneraciones.columns.str.strip

In [ ]:
print(remuneraciones.columns)

## Procesamiento y transformación de datos

1. Se convirtió el período a una variable numérica continua, útil para que el modelo interprete la evolución temporal y mejore la precisión.

In [ ]:
# Transformacion de la columna Periodo
meses = {'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
         'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12}

remuneraciones[['Mes', 'Año']] = remuneraciones['Periodo'].str.split('-', expand=True)
remuneraciones['Mes'] = remuneraciones['Mes'].map(meses)
remuneraciones['Año'] = remuneraciones['Año'].astype(int) + 1900
remuneraciones['FechaNum'] = remuneraciones['Año'] + remuneraciones['Mes'] / 12

2. Se codificó Género para convertirla en variable numérica porque los modelos de machine learning no trabajan con texto.

In [ ]:
# Mujeres = 0, Varones = 1
le = LabelEncoder()
remuneraciones['Genero'] = le.fit_transform(remuneraciones['Genero'])  

3. Industria se convirtió a variable binaria, una por cada sector. Esto permite que el modelo perciba el sector económico.

In [ ]:
remuneraciones = pd.get_dummies(remuneraciones, columns=['Industria'], drop_first=True)

In [ ]:
#Separar X e y
X = remuneraciones.drop(columns=['Periodo', 'Genero', 'Mes', 'Año'])
y = remuneraciones['Genero']

4. Se escaló Valor y FechaNum para que tengan igual peso. Esto es importante porque los modelos como Random Forest no se ven afectados, pero otros sí lo necesitan.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train[['Valor', 'FechaNum']] = scaler.fit_transform(X_train[['Valor', 'FechaNum']])
X_test[['Valor', 'FechaNum']] = scaler.transform(X_test[['Valor', 'FechaNum']])

In [ ]:
# Etapa de prediccion y evaluacion del modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
# Matriz de confusion
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión")
plt.show()

Los resultados arrojaron:
- 2503 (Verdaderos Negativos - TN): El modelo predijo 0 y el valor real era 0.

- 2649 (Verdaderos Positivos - TP): El modelo predijo 1 y el valor real era 1.

- 1128 (Falsos Positivos - FP): El modelo predijo 1 pero el valor real era 0.

- 1062 (Falsos Negativos - FN): El modelo predijo 0 pero el valor real era 1.

Se concluye que: 

- El modelo intenta aprender patrones en los salarios según sector y período para adivinar si la persona es hombre o mujer.

- El hecho de que logre una precisión del 70% sugiere que hay una brecha salarial y/o patrones diferenciados entre géneros que el modelo está captando, pero no de forma determinista (no puede distinguir con certeza absoluta).

- El modelo tiene una precisión general del 70% aproximadamente. El desempeño es bastante equilibrado entre ambas clases, aunque hay una cantidad no menor de errores (más de 1000 falsos positivos y más de 1000 falsos negativos). Se podría explorar mejoras ajustando el umbral de decisión o probando otro modelo.


## IMPORTANTES CONSIDERACIONES ÉTICAS
- ATENCIÓN: predecir el género a partir de variables como salario puede reforzar estereotipos o sesgos estructurales. Esta tarea es útil si se está haciendo un análisis crítico o exploratorio, pero no debería usarse para tomar decisiones reales sobre personas. El equipo decidió desarrollar este modelo de clasificación a fin de poder aplicar el contenido y poder darle un cierre a la propuesta inicial planteada.